<a href="https://colab.research.google.com/github/cxbxmxcx/GenReality/blob/master/GEN_8_DeOldify_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
cd DeOldify

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
!pip install -r colab_requirements.txt

In [ ]:
#@title DEFINE DATASETS
import os
import time
import datetime

dataset_name = "video-bw" #@param ["video-bw"]

datasets = { "video-bw" : 
            { "url" : 'https://www.dropbox.com/s/3u7f2kh48gwje80/video-bw.zip?dl=1',
                          }           
            }

vid_root_folder = '/content/video'
os.makedirs(vid_root_folder, exist_ok=True)
video_folder = vid_root_folder + "/%s" % dataset_name 
color_video_folder = vid_root_folder + "/colorized-%s" % dataset_name
print(f"Video data folders constructed {video_folder}")
os.makedirs(video_folder, exist_ok=True)


In [ ]:
#@title DOWNLOAD VIDEO
if len(os.listdir(video_folder)) == 0:
    from io import BytesIO
    from urllib.request import urlopen
    from zipfile import ZipFile
    zipurl = datasets[dataset_name]['url']
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:        
            zfile.extractall(vid_root_folder)
            print(f"Downloaded & Extracted {zipurl}")

In [ ]:
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

In [ ]:
!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

In [ ]:
colorizer = get_video_colorizer()

In [ ]:
#@title IMAGE SELECTION  { run: "auto" }
import glob
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

files = sorted(glob.glob("%s/*.mp4" % video_folder))
file_idx = 3 #@param {type:"slider", min:0, max:12, step:1}

show_video_in_notebook(files[file_idx])
video = colorizer.colorize_from_file_name(files[file_idx])  
show_video_in_notebook(video) 